In [1]:
import plotly 
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.io as pio
import os

# for ploting online plots, account info is needed
# plotly.tools.set_credentials_file(username='ariellilili', api_key='X8MtW4k2mpBL5tUUHZdi')

# create offline plots
plotly.offline.init_notebook_mode(connected=True)

# import data
SKU = pd.read_csv('SKU_year.csv')

# change timestamp datatype to datetime
SKU['date'] = pd.to_datetime(SKU['date'])
SKU.head()

,date,zoneID,SKU,try_num,transaction_num,unit_profit,profit_day_sum
0,2018-01-01,进门区,4D磁悬浮,213,50,1828,91400
1,2018-01-01,进门区,月光宝盒,202,19,1568,29792
2,2018-01-01,中区1,乳胶,60,9,1556,14004
3,2018-01-01,中区1,椰棕,46,2,1371,2742
4,2018-01-01,中区1,软硬两面,119,3,1140,3420


In [35]:
# time tab
# profit per Q
df = SKU
profit_year = str(int(sum(df['profit_day_sum'])/10000))
df.index = df['date']
df1 = df['profit_day_sum'].resample('Q').sum().to_frame()
df2 = df1.pct_change()

data = [go.Scatter(
    x=['一','二','三','四'],
    y=df1['profit_day_sum'],
    name='利润',
            marker=dict(color='#4ea1d3')
),
    go.Bar(
       x=['一','二','三','四'],
    y=df2['profit_day_sum'],
           yaxis='y2',
           name='环比增长',
    marker=dict(color='#d8e9ef'),
        opacity=0.6
       )]
layout = go.Layout(
    title='2018各季度利润，年度总和='+profit_year+'万元',
    yaxis=dict(
        title='利润'
    ),
    yaxis2=dict(
        title='环比增长',
        tickformat=".1%",
        overlaying='y',
        side='right'
    ),
    legend=dict(orientation="h"),
    font=dict(family='微软雅黑')
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)
pio.write_image(fig, '季度利润.jpeg', scale=5, width=600, height = 400)

In [36]:
# time tab
# number of transactions per month

df = SKU
transaction_year = str(int(sum(df['transaction_num'])))
df.index = df['date']
df1 = df['transaction_num'].resample('M',label='left').sum().to_frame()
df2 = df1.pct_change()

data = [go.Scatter(
    x=df1.index,
    y=df1['transaction_num'],
    name='销量',
            marker=dict(color='#4ea1d3')
),
    go.Bar(
       x=df2.index,
    y=df2['transaction_num'],
           yaxis='y2',
           opacity=0.6,
           name='环比增长',
    marker=dict(color='#d8e9ef')
       )]

layout = go.Layout(
    title='2018月销量，年度总和='+transaction_year+'笔',
    yaxis=dict(
        title='销量'
    ),
    yaxis2=dict(
        title='环比增长',
        tickformat=".1%",
        overlaying='y',
        side='right'
    ),
    legend=dict(orientation="h"),
    font=dict(family='微软雅黑')
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)
pio.write_image(fig, '月销量.jpeg', scale=5, width=600, height = 400)

In [37]:
# SKU
# profit per SKU unit
df = SKU['unit_profit'].groupby(SKU['SKU']).mean().to_frame().sort_values(by=['unit_profit'],ascending=False)
data = [go.Bar(
    x=df.index,
    y=df['unit_profit'],
    marker=dict(color='#4ea1d3')
)]
layout = go.Layout(
    title='单件平均利润（元）',
    font=dict(family='微软雅黑')
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)
pio.write_image(fig, '单件利润.jpeg', scale=5, width=600, height = 400)

In [38]:
# SKU
# profit per SKU unit
df = SKU['transaction_num'].groupby(SKU['SKU']).sum().to_frame().sort_values(by=['transaction_num'],ascending=False)
data = [go.Bar(
    x=df.index,
    y=df['transaction_num'],
    marker=dict(color='#4ea1d3')
)]
layout = go.Layout(
    title='床垫年销量（件）',
    font=dict(family='微软雅黑')
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)
pio.write_image(fig, '单件销量.jpeg', scale=5, width=600, height = 400)

In [3]:
# try vs transaction
df = SKU[['try_num','transaction_num']].groupby(SKU['SKU']).mean().sort_values(by=['try_num'],ascending=False).round(1)
df['conversion'] = df['transaction_num']/df['try_num']

trace1 = go.Bar(
    x=df.index,
    y=df['try_num'],
    name='试睡次数',
    marker=dict(color='#4ea1d3')
)
trace2 = go.Bar(
    x=df.index,
    y=df['transaction_num'],
    name='交易量',
    marker=dict(color='#e85a71')
)
trace3 = go.Scatter(
    x=df.index,
    y=df['conversion'],
    name='转化率',
    yaxis='y2',
    marker=dict(color='#454552')
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group',
    title='日均试睡次数vs交易量',
    yaxis=dict(
        title='试睡次数、交易量'
    ),
    yaxis2=dict(
        title='转化率',
        tickformat=".0%",
        overlaying='y',
        side='right'
    ),
    legend=dict(orientation="h"),
    font=dict(family='微软雅黑')
    
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)
pio.write_image(fig, '试睡和销量.jpeg', scale=5, width=1200, height = 400)

In [24]:

# profit per zone
df = SKU[['profit_day_sum','transaction_num']].groupby(SKU['zoneID']).sum()
df['profit_day_sum'] = (df['profit_day_sum']/10000).round(1)

trace1 = go.Bar(
    x=df.index,
    y=df['profit_day_sum'],
    name='利润'
)
trace2 = go.Scatter(
    x=df.index,
    y=df['transaction_num'],
    name='交易量',
    yaxis='y2'
)


data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title='各区域总利润及交易量',
    yaxis=dict(
        title='总利润（万元）'
    ),
    yaxis2=dict(
        title='交易量（笔）',
        overlaying='y',
        side='right'
    ),
    legend=dict(orientation="h"),
    font=dict(family='微软雅黑')
    
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [40]:
# profit per zone
df = SKU[['transaction_num']].groupby(SKU['zoneID']).mean()
df['ppl'] = [1200, 1100, 2000, 800]
df['ppl_per_trans'] = df['ppl']/df['transaction_num']
colors = ['#e85a71','#4ea1d3','#4ea1d3','#4ea1d3']
p1 = go.Scatter(x=df['transaction_num'], 
                y=df['ppl'],
                text=df.index,
               mode='markers+text',
                hoverinfo='text',
               marker=dict(
                   size=df['ppl_per_trans']/3,
                   color = colors))

layout = go.Layout(title='各区域日均人流量vs销量（圆圈大小=人流量/销量）',
                   yaxis=dict(title='人流量'),
                  xaxis=dict(title='销量'),
                  font=dict(family='微软雅黑'))

fig = go.Figure(data=[p1], layout=layout)
plotly.offline.iplot(fig)
pio.write_image(fig, '各区表现.jpeg', scale=5, width=1200, height = 400)